In [51]:
import numpy as np
import pandas as pd
import os
import pickle
import math
import pdb
from matplotlib import pyplot as plt
import importlib
from scipy.optimize import fsolve

from cartopy import crs as ccrs
from cartopy import feature as cfeature
from matplotlib import cm

import loaders
import geography
import visualization

importlib.reload(loaders)
importlib.reload(geography)
importlib.reload(visualization)
import visualization as vis

%matplotlib
specs = loaders.load_density()
specs = geography.get_coordinate_functions(specs)
airports = loaders.load_airports(specs)
travel = loaders.load_travel(airports) 
valid = airports.NodeName.values
travel = {k:v.query('Origin in @valid and Dest in @valid') for k, v in travel.items()}
    
airports = loaders.calculate_outbreaks(airports, kappa=1, n=1)
travel = {k:loaders.augment_travel(v, airports) for k, v in travel.items()}
    

Using matplotlib backend: Qt5Agg


In [100]:
month = 12#'annual'
destinations = travel[month].query('Origin == "WUH"')
destinations = destinations.assign(Month=month)
drop = ['row_max', 'row_top', 'row_bottom', 'col_left', 'Lon', 'Lat', 'row', 'col', 'horz_window', 'vert_window_avg']
drop = drop + ['lon_max', 'col_max', 'lat_max', 'vert_window', 'horz_window_avg', 'idx', 'vert_pixel', 'horz_pixel']
drop = drop + ['col_right', 'shape', 'NodeName', 'origin_lat', 'origin_lon', 'dest_lon', 'dest_lat', 'upper', 'lower']
cut = airports.loc[destinations.Dest.values].sort_values('density').drop(drop, axis=1, errors='ignore')
destinations = destinations.drop(drop, axis=1, errors='ignore')
destinations = destinations.assign(desc=airports.loc[destinations.Dest, 'OAGName'].values)

destinations = destinations.sort_values('Prediction')
#destinations['Prediction'] = destinations.Prediction // 100
destinations.reset_index(drop=True).to_csv(f'wuhan_{month}.csv', sep ='\t')

In [175]:
ap = pd.read_csv('./data/airports.csv').query('iso_country == "IN"')
ap.iata_code.dropna().unique()
#africa_airports = ap.query('continent == "AF"')
#africa_airports

array(['VDY', 'JGB', 'NVY', 'RJI', 'TEI', 'OMN', 'REW', 'DIU', 'AMD',
       'AKD', 'IXU', 'BOM', 'PAB', 'BHJ', 'IXG', 'BDQ', 'BHO', 'BHU',
       'NMB', 'GUX', 'GOI', 'HBX', 'IDR', 'JLR', 'JGA', 'IXY', 'HJR',
       'KLH', 'IXK', 'NDC', 'NAG', 'ISK', 'PNQ', 'PBD', 'RTC', 'RAJ',
       'RPR', 'SAG', 'SSE', 'STV', 'UDR', 'IXV', 'IXA', 'IXB', 'RGH',
       'SHL', 'BBI', 'CCU', 'COH', 'DBD', 'RDP', 'DEP', 'GOP', 'GAU',
       'GAY', 'IMF', 'PYB', 'IXW', 'JRH', 'IXQ', 'IXH', 'IXS', 'IXN',
       'AJL', 'IXI', 'LDA', 'DIB', 'DMU', 'MZU', 'IXT', 'PAT', 'IXR',
       'RRK', 'RUP', 'TEZ', 'VTZ', 'ZER', 'AGR', 'IXD', 'ATQ', 'AIP',
       'BKB', 'VNS', 'KUU', 'BUP', 'BEK', 'IXC', 'KNU', 'DED', 'DEL',
       'DHM', 'GWL', 'HSS', 'JDH', 'JAI', 'JSA', 'IXJ', 'KQH', 'KTU',
       'LUH', 'IXL', 'LKO', 'IXP', 'PGH', 'SLV', 'SXR', 'TNI', 'LTU',
       'TCR', 'AGX', 'BEP', 'BLR', 'VGA', 'CJB', 'COK', 'CCJ', 'CDP',
       'CBD', 'HYD', 'BPM', 'CNN', 'IXM', 'IXE', 'MAA', 'MYQ', 'IXZ',
       'PNY', 'PUT',